In [1]:
import pandas as pd
import numpy as np

## Read Data

In [2]:
data = pd.read_csv('./Data/PURCHASE ORDER DATA EXTRACT 2012-2015_0.csv')

In [3]:
data.columns

Index(['Creation Date', 'Purchase Date', 'Fiscal Year', 'LPA Number',
       'Purchase Order Number', 'Requisition Number', 'Acquisition Type',
       'Sub-Acquisition Type', 'Acquisition Method', 'Sub-Acquisition Method',
       'Department Name', 'Supplier Code', 'Supplier Name',
       'Supplier Qualifications', 'Supplier Zip Code', 'CalCard', 'Item Name',
       'Item Description', 'Quantity', 'Unit Price', 'Total Price',
       'Classification Codes', 'Normalized UNSPSC', 'Commodity Title', 'Class',
       'Class Title', 'Family', 'Family Title', 'Segment', 'Segment Title',
       'Location'],
      dtype='object')

In [4]:
data['Creation Date']

0         08/27/2013
1         01/29/2014
2         11/01/2013
3         06/13/2014
4         03/12/2014
             ...    
346013    11/03/2014
346014    08/14/2014
346015    05/09/2014
346016    01/14/2014
346017    10/09/2013
Name: Creation Date, Length: 346018, dtype: object

In [5]:
data['Purchase Date']

0                NaN
1                NaN
2                NaN
3         06/05/2014
4         03/12/2014
             ...    
346013    09/25/2014
346014    07/20/2011
346015           NaN
346016    01/14/2014
346017    09/09/2013
Name: Purchase Date, Length: 346018, dtype: object

In [27]:
# !pip install transformers
# !pip install --user trl
# !pip install transformers[torch]
# !pip install accelerate -U

  Using cached trl-0.7.7-py3-none-any.whl (139 kB)
  Using cached tyro-0.6.2-py3-none-any.whl (78 kB)
  Using cached docstring_parser-0.15-py3-none-any.whl (36 kB)
  Using cached shtab-1.6.5-py3-none-any.whl (13 kB)
  Using cached rich-13.7.0-py3-none-any.whl (240 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [8]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [10]:
# from trl import SFTTrainer

In [12]:
# from transformers import AutoModelForCausalLM
from datasets import load_dataset
# from trl import SFTTrainer

# dataset = load_dataset("imdb", split="train")

# model = AutoModelForCausalLM.from_pretrained("bert-base-cased")

# trainer = SFTTrainer(
#     model,
#     train_dataset=dataset,
#     dataset_text_field="text",
#     max_seq_length=512,
# )

In [13]:
# trainer.train()

In [14]:
# dataset

In [15]:
from datasets import Dataset

In [16]:
ind_mask = np.random.rand(*data.shape)<0.1

In [17]:
data_text = []
column_name = data.columns.to_list()
for i, row in data.iterrows():
#     if i<=5:
    cols = ''
    output = ''
    inputs = ''
    for j,c in enumerate(column_name):
        if ind_mask[i,j]:
            cols = cols + f'{c},'
            output = output + f'{row.loc[c]}, '
        else:
            inputs = inputs + f'{c} {row.loc[c]} '

    text = ("Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n\n"
        "### Instruction:\nwhat are the values of {}\n\n### Input:\n{}\n\n### Response:\n{}").format(cols,inputs,output)
    data_text.append(text)

In [28]:
print(data_text[100])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
what are the values of Sub-Acquisition Type,Supplier Qualifications,Normalized UNSPSC,Class,

### Input:
Creation Date 08/26/2014 Purchase Date 08/22/2014 Fiscal Year 2014-2015 LPA Number nan Purchase Order Number C440-5063 Requisition Number nan Acquisition Type NON-IT Goods Acquisition Method Informal Competitive Sub-Acquisition Method nan Department Name Statewide Health Planning & Development, Office of Supplier Code 1033180.0 Supplier Name Franklin Young International, Inc. Supplier Zip Code 91784 CalCard NO Item Name post-it notes Item Description post it notes Quantity 25.0 Unit Price $6.43 Total Price $160.75 Classification Codes 14111829 Commodity Title nan Class Title nan Family nan Family Title nan Segment nan Segment Title nan Location 91784
(34.137067, -117.659127) 

### Response:
nan, CA-DVBE CA-MB 

In [19]:
len(data_text)

346018

In [20]:
dataset = Dataset.from_pandas(data)

In [21]:
dataset = dataset.add_column(name='text', column=data_text)

In [22]:
dataset['text']

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nwhat are the values of Acquisition Type,\n\n### Input:\nCreation Date 08/27/2013 Purchase Date nan Fiscal Year 2013-2014 LPA Number 7-12-70-26 Purchase Order Number REQ0011118 Requisition Number REQ0011118 Sub-Acquisition Type nan Acquisition Method WSCA/Coop Sub-Acquisition Method nan Department Name Consumer Affairs, Department of Supplier Code 1740272.0 Supplier Name Pitney Bowes Supplier Qualifications nan Supplier Zip Code nan CalCard NO Item Name USB Item Description USB Quantity 1.0 Unit Price $1.00 Total Price $1.00 Classification Codes nan Normalized UNSPSC nan Commodity Title nan Class nan Class Title nan Family nan Family Title nan Segment nan Segment Title nan Location nan \n\n### Response:\nIT Goods, ',
 'Below is an instruction that describes a task, paired with an input that provides further c

In [23]:
dataset = dataset.train_test_split(test_size = 0.01)

In [24]:
dataset['train'].to_csv('./Data/hugging_face_train.csv')

Creating CSV from Arrow format:   0%|          | 0/343 [00:00<?, ?ba/s]

569032031

In [25]:
dataset['test'].to_csv('./Data/hugging_face_test.csv')

Creating CSV from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

5754926